# SMBO Gaussian Process - New York City Taxi

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import seaborn as sns
import pandas as pd

import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score

sns.set()

ModuleNotFoundError: No module named 'bayes_opt'

## Chargement des données

In [ ]:
data = pd.read_csv("nyc_taxi.csv").iloc[:, 1:]
data.head()

In [3]:
data['pickup_datetime'] = data['pickup_datetime'].str.slice(0, 16)
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:00+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:00+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:00+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


## Nettoyage des données

In [4]:
# Remove observations with missing values
# Since there are only a few of these, i'm not concerned with imputation
data.dropna(how='any', axis='rows', inplace=True)

# Removing observations with erroneous values
mask = data['pickup_longitude'].between(-75, -73)
mask &= data['dropoff_longitude'].between(-75, -73)
mask &= data['pickup_latitude'].between(40, 42)
mask &= data['dropoff_latitude'].between(40, 42)
mask &= data['passenger_count'].between(0, 8)
mask &= data['fare_amount'].between(0, 250)

data = data[mask]

## Feature Engineering

In [5]:
def dist(pickup_lat, pickup_long, dropoff_lat, dropoff_long):  
    distance = np.abs(dropoff_lat - pickup_lat) + np.abs(dropoff_long - pickup_long)
    return distance

In [6]:
def transform(data):
    # Extract date attributes and then drop the pickup_datetime column
    data['hour'] = data['pickup_datetime'].dt.hour
    data['day'] = data['pickup_datetime'].dt.day
    data['month'] = data['pickup_datetime'].dt.month
    data['year'] = data['pickup_datetime'].dt.year
    data = data.drop('pickup_datetime', axis=1)

    # Distances to nearby airports, and city center
    # By reporting distances to these points, the model can somewhat triangulate other locations of interest
    nyc = (-74.0063889, 40.7141667)
    jfk = (-73.7822222222, 40.6441666667)
    ewr = (-74.175, 40.69)
    lgr = (-73.87, 40.77)
    data['distance_to_center'] = dist(nyc[1], nyc[0],
                                      data['pickup_latitude'], data['pickup_longitude'])
    data['pickup_distance_to_jfk'] = dist(jfk[1], jfk[0],
                                         data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_jfk'] = dist(jfk[1], jfk[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_ewr'] = dist(ewr[1], ewr[0], 
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_ewr'] = dist(ewr[1], ewr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_lgr'] = dist(lgr[1], lgr[0],
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_lgr'] = dist(lgr[1], lgr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    
    data['long_dist'] = data['pickup_longitude'] - data['dropoff_longitude']
    data['lat_dist'] = data['pickup_latitude'] - data['dropoff_latitude']
    
    data['dist'] = dist(data['pickup_latitude'], data['pickup_longitude'],
                        data['dropoff_latitude'], data['dropoff_longitude'])
    
    return data


dataset = transform(data)
dataset = dataset.drop(['key'], axis=1)
dataset = dataset.sample(10000)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset.drop('fare_amount', axis=1),
                                                    dataset['fare_amount'], test_size=0.25)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

/opt/conda/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [8]:
dataset.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day,month,year,distance_to_center,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,long_dist,lat_dist,dist
590359,7.3,-73.989373,40.723242,-74.010693,40.713497,5,22,2,8,2010,0.026091,0.286226,0.297801,0.218869,0.187804,0.166131,0.197196,0.021320,0.009745,0.031065
587045,9.7,-74.005965,40.748707,-73.980765,40.763633,4,14,12,1,2009,0.034964,0.328283,0.318009,0.227742,0.267868,0.157258,0.117132,-0.025200,-0.014926,0.040126
352067,8.5,-73.982462,40.767397,-73.979372,40.784540,1,15,24,3,2014,0.077157,0.323470,0.337523,0.269935,0.290168,0.115065,0.123912,-0.003090,-0.017143,0.020233
637678,4.9,-73.982383,40.774613,-74.004048,40.725588,1,20,17,9,2010,0.084452,0.330607,0.303247,0.277230,0.206540,0.116996,0.178460,0.021665,0.049025,0.070690
853464,8.5,-73.984154,40.755379,-73.978722,40.777100,2,21,18,4,2015,0.063447,0.313144,0.329432,0.256225,0.283378,0.128775,0.115821,-0.005432,-0.021721,0.027153


## Optimisation bayésienne des hyper-paramètres

### Minimisation de la fonction de perte

In [41]:
def xgb_evaluate(max_depth, gamma, colsample_bytree, nrounds):
    params = {'eval_metric': 'rmse',
              'objective':'reg:squarederror',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
              'nrounds': int(nrounds)}
    
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3, metrics=['rmse'])    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -cv_result['test-rmse-mean'].iloc[-1]

In [43]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9),
                                             'nrounds': (100, 1000)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=3, n_iter=10, acq='ei')

|   iter    |  target   | colsam... |   gamma   | max_depth |  nrounds  |
-------------------------------------------------------------------------
|  1        | -4.395    |  0.6441   |  0.777    |  6.344    |  728.6    |
|  2        | -4.39     |  0.447    |  0.3536   |  3.225    |  141.3    |
|  3        | -4.375    |  0.7995   |  0.4795   |  4.51     |  682.7    |
|  4        | -4.394    |  0.9      |  0.0      |  3.0      |  448.3    |
|  5        | -4.394    |  0.9      |  7.366e-0 |  3.0      |  606.7    |
|  6        | -4.447    |  0.7624   |  0.9814   |  6.919    |  280.9    |
|  7        | -4.417    |  0.9      |  1.0      |  7.0      |  100.0    |
|  8        | -4.394    |  0.9      |  0.0      |  3.0      |  1e+03    |
|  9        | -4.36     |  0.8848   |  0.3678   |  3.021    |  916.5    |
|  10       | -4.372    |  0.4089   |  0.8275   |  6.994    |  931.4    |
|  11       | -4.331    |  0.5103   |  0.09453  |  3.021    |  879.5    |
|  12       | -4.335    |  0.3973   | 

## Entraînement du modèle optimisé

In [50]:
model = xgb.XGBRegressor(**{'eval_metric': 'rmse',
              'objective':'reg:squarederror',
              'max_depth': 3,
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': 0.1,
              'colsample_bytree': 0.5,
              'nrounds': 1000})

model.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, eta=0.1,
             eval_metric='rmse', gamma=0.1, importance_type='gain',
             learning_rate=0.1, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
             nrounds=1000, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [51]:
from sklearn.metrics import r2_score

print(r2_score(y_test, model.predict(X_test)))

0.7605249573374324
